# A first stab: DQN

[DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) is a classical RL algorithm which should provide a nice baseline for further work.

Classical RL techniques woul probably not work very well without further feature engineering, because the current state space is quite large.

In [1]:
import tianshou as ts 
from tianshou.utils import TensorboardLogger

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

import numpy as np

import os

In [2]:
from utils_preprocess import compute_frame_features, compute_foa_features

from env_markov import MarkovGazeEnv

/Users/johndoe/Desktop/uni/affectivecompute/project/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists


## Data and environment initialisation

In [3]:
vid_filename = "001"
mat_filename = vid_filename + ".mat"
target_subject = 0

In [4]:
patch_bounding_boxes_per_frame, patch_centres_per_frame, speaker_info_per_frame = compute_frame_features(
    vid_filename
)

foa_centres_per_frame_per_subject, patch_weights_per_frame = compute_foa_features(
    mat_filename, patch_centres_per_frame
)
foa_centres_per_frame = [frame[target_subject] for frame in foa_centres_per_frame_per_subject]

In [5]:
env = MarkovGazeEnv(
    patch_bounding_boxes_per_frame,
    patch_centres_per_frame,
    speaker_info_per_frame,
    foa_centres_per_frame,
    patch_weights_per_frame,
)

In [6]:
# env.observation_space.sample(), env.action_space.sample()

For efficiency, it's a good idea to set up some vectorized environments.

In [7]:
num_train_envs = 5
num_test_envs = 10

train_envs = ts.env.DummyVectorEnv([lambda: env for _ in range(num_train_envs)])
test_envs = ts.env.DummyVectorEnv([lambda: env for _ in range(num_test_envs)])

## DQN

First, let's construct the network.

The biggest headache comes from the observations: they're quite complex. So, we build multiple networks, each processing a part of an observation and combining their outputs in the end!

In [8]:
class Net(nn.Module):
    def __init__(self, observation_space, action_shape):
        super().__init__()

        self.num_patches = observation_space['patch_centres'].shape[0]

        # network for patch_centres
        self.patch_centres_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['patch_centres'].shape), 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True)
        )

        # network for patch_bounding_boxes
        self.patch_bboxes_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['patch_bounding_boxes'].shape), 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True)
        )

        # network for speaker_info
        self.speaker_info_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['speaker_info'].shape), 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 32),
            nn.ReLU(inplace=True)
        )

        # combining the outputs of all networks
        self.combined_net = nn.Sequential(
            nn.Linear(64 + 64 + 32, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape))
        )

    def forward(self, obs, state=None, info={}):
        patch_centres = torch.tensor(obs['patch_centres'], dtype=torch.float32)
        patch_bboxes = torch.tensor(obs['patch_bounding_boxes'], dtype=torch.float32)
        speaker_info = torch.tensor(obs['speaker_info'], dtype=torch.float32)

        patch_centres = patch_centres.view(patch_centres.size(0), -1)
        patch_bboxes = patch_bboxes.view(patch_bboxes.size(0), -1)
        speaker_info = speaker_info.view(speaker_info.size(0), -1)

        # pass through respective networks
        patch_centres_out = self.patch_centres_net(patch_centres)
        patch_bboxes_out = self.patch_bboxes_net(patch_bboxes)
        speaker_info_out = self.speaker_info_net(speaker_info)

        # combine outputs
        combined = torch.cat([patch_centres_out, patch_bboxes_out, speaker_info_out], dim=1)

        logits = self.combined_net(combined)

        return logits, state

In [9]:
state_shape = env.observation_space
action_shape = env.action_space.n

net = Net(state_shape, action_shape)
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [10]:
net

Net(
  (patch_centres_net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (patch_bboxes_net): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (speaker_info_net): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU(inplace=True)
  )
  (combined_net): Sequential(
    (0): Linear(in_features=160, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

## Setting up DQN

First, we need to set up the policy, which is readily done in Tianshou.

In [11]:
policy = ts.policy.DQNPolicy(
    model=net, 
    optim=optim, 
    discount_factor=0.99,
    estimation_step=1,
    target_update_freq=50
)

Then, we need to set up the collectors, i.e., the objects that will be interacting with the environment according to the above policy and collect the generated data.

In classical DQN fashion, we store the data in a replay buffer.

In [12]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(6000, num_train_envs))

test_collector = ts.data.Collector(policy, test_envs)

## Training

In [13]:
num_epochs = 20
num_steps_per_epoch = 3000
step_per_collect = 10 # update the Q-values at each step
episode_per_test = 5 # use 5 episodes to test the policy
batch_size = 30 # one second of data (videos are at 30FPS)

log_path = os.path.join("logs", "dqn")
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)

In [14]:
result = ts.trainer.offpolicy_trainer(
    policy, 
    train_collector, 
    test_collector,
    max_epoch=num_epochs,
    step_per_epoch=num_steps_per_epoch,
    step_per_collect=step_per_collect,
    episode_per_test=episode_per_test,
    batch_size=batch_size,
    logger=logger,
)

Epoch #1: 3001it [00:08, 368.63it/s, env_step=3000, len=120, loss=6.392, n/ep=2, n/st=10, rew=77.29]                          


Epoch #1: test_reward: 230.130769 ± 181.954642, best_reward: 230.130769 ± 206.245826 in #0


Epoch #2: 3001it [00:08, 368.25it/s, env_step=6000, len=120, loss=8.135, n/ep=2, n/st=10, rew=73.72]                           


Epoch #2: test_reward: 230.130769 ± 186.879228, best_reward: 230.130769 ± 206.245826 in #0


Epoch #3: 3001it [00:08, 370.53it/s, env_step=9000, len=120, loss=11.961, n/ep=2, n/st=10, rew=56.41]                          


...

Epoch #18: test_reward: 230.130769 ± 192.108256, best_reward: 230.130769 ± 206.245826 in #0


Epoch #19: 3001it [00:08, 368.97it/s, env_step=57000, len=120, loss=13.133, n/ep=2, n/st=10, rew=65.03]                          


Epoch #19: test_reward: 230.130769 ± 170.978114, best_reward: 230.130769 ± 206.245826 in #0


Epoch #20: 3001it [00:08, 363.16it/s, env_step=60000, len=120, loss=18.336, n/ep=2, n/st=10, rew=76.37]                          


Epoch #20: test_reward: 230.130769 ± 189.504165, best_reward: 230.130769 ± 206.245826 in #0


In [15]:
result

{'duration': '168.85s',
 'train_time/model': '154.27s',
 'test_step': 37779,
 'test_episode': 105,
 'test_time': '5.49s',
 'test_speed': '6882.52 step/s',
 'best_reward': 230.13076861072403,
 'best_result': '230.13 ± 206.25',
 'train_step': 60000,
 'train_episode': 200,
 'train_time/collector': '9.09s',
 'train_speed': '367.28 step/s'}

Well, that's quite a let down...

Although, there's not much to be surprised about: there is so very little information passed to the networks! 

Plus, I'm still not too sure that the problem is really amenable to RL in the first place.

At this point, I have two choices:
1. I keep fine-tuning hyperparameters until I get an acceptable result,
2. I try a different approach.

I'll opt for the second option, but I made this code into a notebook precisely because, that way, tinkering would be easier. So, if you wish to tune and fine-tune things, go ahead!

In [16]:
policy.eval()
policy.set_eps(0.05)

collector = ts.data.Collector(policy, train_envs)
collector.collect(n_episode=10)

{'n/ep': 10,
 'n/st': 3000,
 'rews': array([ 91.27054762,  77.12606428,  67.49008293,  77.55667718,
         95.07588698,  56.58696209, 329.79185276,  88.81977665,
        480.1971609 , 551.34139371]),
 'lens': array([120, 120, 120, 120, 120, 120, 510, 150, 810, 810]),
 'idxs': array([3, 4, 3, 4, 3, 4, 2, 4, 0, 1]),
 'rew': 191.5256405089367,
 'len': 300.0,
 'rew_std': 179.30079631351055,
 'len_std': 279.4995527724508}

It does perform quite well, every once in a while...

### TensorBoard visualisation

In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [19]:
%tensorboard --logdir logs/dqn

Reusing TensorBoard on port 6006 (pid 5009), started 0:02:14 ago. (Use '!kill 5009' to kill it.)